## READ ME

Name: Kim, Minjun

Student ID: 20195024

Instructions: `HW4 Code.ipynb` contains codes of Q5~Q8. For the code of Q9, mini-competition of kaggle, I saved it separately as a file named `HW4 Code_mini_kaggle.ipynb`.

In [114]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

| Columns    | Description                                         |
|------------|-----------------------------------------------------|
| id         | Unique identifier of the available flight option   |
| date       | Departure date (YYYY-MM-DD)                         |
| airline    | The name of the airline company                     |
| ch_code    | The flight code character                           |
| num_code   | The flight code number                              |
| dep_time   | Departure time (hh:mm)                              |
| from       | City of departure                                   |
| time_taken | Time it takes to travel between cities              |
| stop       | The number of stopover in a journey                 |
| arr_time   | Arrival time (hh:mm)                                |
| to         | City of arrival                                     |
| class      | Seat class (Economy or Business)                    |
| price      | Target variable: Available ticket price             |

The dataset contains the available flights captured by a travel agency in Feb 10th, 2022.

In [102]:
train = pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
submission=pd.read_csv('submission.csv')

In [103]:
train.head()

,id,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,class,price
0,UVYA93,2022-02-11,Indigo,6E,2106,03:00,Delhi,02h 10m,non-stop,05:10,Kolkata,economy,6270
1,1Y6UUB,2022-02-11,AirAsia,I5,764,04:25,Delhi,10h 20m,1-stop,14:45,Bangalore,economy,7423
2,J62B2K,2022-02-11,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,economy,5956
3,HW31QF,2022-02-11,AirAsia,I5,548,04:45,Delhi,02h 25m,non-stop,07:10,Kolkata,economy,6060
4,AI4BTM,2022-02-11,AirAsia,I5,548,04:45,Delhi,14h 20m,1-stop,19:05,Bangalore,economy,21343


In [104]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243538 entries, 0 to 243537
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          243538 non-null  object
 1   date        243538 non-null  object
 2   airline     243538 non-null  object
 3   ch_code     243538 non-null  object
 4   num_code    243538 non-null  int64 
 5   dep_time    243538 non-null  object
 6   from        243538 non-null  object
 7   time_taken  243538 non-null  object
 8   stop        243538 non-null  object
 9   arr_time    243538 non-null  object
 10  to          243538 non-null  object
 11  class       243538 non-null  object
 12  price       243538 non-null  int64 
dtypes: int64(2), object(11)
memory usage: 24.2+ MB


There are no null values in the trainset

In [105]:
train.nunique() / len(train)

id            1.000000
date          0.000164
airline       0.000033
ch_code       0.000033
num_code      0.004369
dep_time      0.001014
from          0.000025
time_taken    0.001955
stop          0.000156
arr_time      0.001068
to            0.000025
class         0.000008
price         0.048773
dtype: float64

all ids are unique, so we don't need to check for duplicates, also don't need to transform dtype to 'category'

In [106]:
test.head()

,id,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,class
0,RB0IF4,2022-03-23,Indigo,6E,5376,00:10,Mumbai,08h 50m,1-stop,09:00,Delhi,economy
1,VZE001,2022-03-23,Indigo,6E,5376,00:10,Mumbai,07h 10m,1-stop,07:20,Delhi,economy
2,KBPR5E,2022-03-23,Indigo,6E,5376,00:10,Mumbai,10h 20m,1-stop,10:30,Delhi,economy
3,P9XC55,2022-03-23,AirAsia,I5,722,00:30,Bangalore,02h 50m,non-stop,03:20,Delhi,economy
4,SWZ04L,2022-03-23,AirAsia,I5,722,00:30,Bangalore,06h 05m,1-stop,06:35,Mumbai,economy


In [107]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56723 entries, 0 to 56722
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          56723 non-null  object
 1   date        56723 non-null  object
 2   airline     56723 non-null  object
 3   ch_code     56723 non-null  object
 4   num_code    56723 non-null  int64 
 5   dep_time    56723 non-null  object
 6   from        56723 non-null  object
 7   time_taken  56723 non-null  object
 8   stop        56723 non-null  object
 9   arr_time    56723 non-null  object
 10  to          56723 non-null  object
 11  class       56723 non-null  object
dtypes: int64(1), object(11)
memory usage: 5.2+ MB


In [108]:
submission.shape

(56723, 2)

In [109]:
#이거 보면 1.01h m 이런거 있음
unique_times = pd.Series(train['time_taken'].unique())
print(unique_times.tail(10))

466    06h 59m
467     5h 30m
468    39h 50m
469    1.01h m
470    1.02h m
471    1.03h m
472    35h 40m
473    35h 20m
474    29h 55m
475    36h 35m
dtype: object


In [110]:
object_features=['airline', 'ch_code', 'from', 'time_taken', 'stop', 'to', 'class']

for i in object_features:
    print(train[i].value_counts())
    print()

airline
Vistara      103599
Air India     66205
Indigo        34550
GO FIRST      18791
AirAsia       12714
SpiceJet       7595
StarAir          48
Trujet           36
Name: count, dtype: int64

ch_code
UK    103599
AI     66205
6E     34550
G8     18791
I5     12714
SG      7595
S5        48
2T        36
Name: count, dtype: int64

from
Delhi        49715
Mumbai       49604
Bangalore    42228
Kolkata      37635
Hyderabad    32982
Chennai      31374
Name: count, dtype: int64

time_taken
02h 10m    3464
02h 15m    3321
02h 45m    2339
02h 05m    2203
02h 50m    1895
           ... 
38h 40m       1
30h 50m       1
47h 05m       1
41h 35m       1
44h 30m       1
Name: count, Length: 476, dtype: int64

stop
1-stop                        197421
non-stop                       29132
2+-stop                        10974
1-stop: Via IXU                 1492
1-stop: Via IDR                 1169
1-stop: Via Patna                541
1-stop: Via PAT                  306
1-stop: Via Indore           

In [111]:
def time_to_minutes(t):
    hours, minutes = map(int, t.split(":"))
    return hours * 60 + minutes


#대부분의 데이터는 24h 00m로 나오지만 1.01h m 과 같은 몇몇 결측치들을 봤음... 따라서 그거 처리하는 함수
def time_taken_to_minutes(t):
    #여분의 공백을 제거하고 소문자로 변환
    t = t.lower().strip()
    
    #"h"로 시간과 분을 분리
    parts = t.split('h')
    hours = 0
    minutes = 0

    #시간 부분 처리
    if parts[0]:
        hours = float(parts[0])
    
    #분 부분 처리
    if len(parts) > 1 and parts[1]:
        minutes_part = parts[1].strip().replace('m', '')  #'m' 문자를 제거
        if minutes_part:  #분 부분이 실제로 숫자를 포함하고 있는지 확인
            minutes = float(minutes_part)
    
    total_minutes = int(hours * 60 + minutes)
    
    return total_minutes

#교수님이 수업시간에 설명하셨듯이, 시간을 삼각함수에 매핑하면 모델 학습에 도움됨
def time_to_sin(t):
    return np.sin(2 * np.pi * time_to_minutes(t) / 1440)
def time_to_cos(t):
    return np.cos(2 * np.pi * time_to_minutes(t) / 1440)

In [112]:
def feat_eng(df):
    df['sin_dep_time'] = df['dep_time'].apply(time_to_sin)
    df['cos_dep_time'] = df['dep_time'].apply(time_to_cos)
    df['sin_arr_time'] = df['arr_time'].apply(time_to_sin)
    df['cos_arr_time'] = df['arr_time'].apply(time_to_cos)

    #1.01h m, #24h 00m 이런 것들 전부 그냥 분으로 바꾸어 준 것을 feature로 저장
    df['time_taken_minutes'] = df['time_taken'].apply(time_taken_to_minutes)
    df['date'] = pd.to_datetime(df['date'])

    #문제 조건에 2022년 2월 10일에 조사한 데이터라고 명시되어 있었음, 따라서 2022년 2월 10일과의 날짜 차이를 구함
    df['days_left'] = (df['date'] - pd.Timestamp('2022-02-10')).dt.days
    
    #더미 변수 더하기
    object_features=['airline', 'ch_code', 'from', 'stop', 'to', 'class']
    df = pd.get_dummies(df, columns=object_features, drop_first=True, dtype=np.int8)
    
    return df  

In [113]:
train = pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
submission=pd.read_csv('submission.csv')

#train set feature engineering
processed_train = feat_eng(train)
X_tr = processed_train.drop(columns=['price', 'id', 'date', 'dep_time', 'arr_time','time_taken'])
y_tr = processed_train['price']

#test set feature engineering
processed_test = feat_eng(test)
X_test = processed_test.drop(columns=['id', 'date', 'dep_time', 'arr_time','time_taken'])
X_test = X_test.reindex(columns=X_tr.columns, fill_value=0)

#Data Scaling 이후 학습
dtr_pipeline = make_pipeline(StandardScaler(), DecisionTreeRegressor(random_state=0))
dtr_pipeline.fit(X_tr, y_tr)
y_pred = dtr_pipeline.predict(X_test)

#Dataframe 만든후, submission.csv 파일로 저장
submission = pd.DataFrame({'id': test['id'], 'price': y_pred})
submission.to_csv('submission.csv', index=False)